In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns

In [2]:
# load dataset
df_order = pd.read_csv("../dataset/order.csv")
df_group = pd.read_csv("../dataset/group.csv")
df_airline = pd.read_csv("../dataset/airline.csv")
df_day_schedule = pd.read_csv("../dataset/day_schedule.csv")
df_train = pd.read_csv("../dataset/training-set.csv")
df_test = pd.read_csv("../dataset/testing-set.csv")

In [3]:
print(df_order.shape)
print(df_group.shape)
print(df_airline.shape)
print(df_day_schedule.shape)
print(df_train.shape)
print(df_test.shape)

(300000, 7)
(35292, 8)
(74659, 6)
(218251, 3)
(146620, 2)
(150400, 2)


In [4]:
# date Conversion

month = {'Jan': '01', 'Feb': '02' , 'Mar':'03' ,'Apr': '04',
'May': '05', 'Jun': '06' , 'Jul': '07' , 'Aug':'08',
'Sep':'09', 'Oct':'10' , 'Nov':'11', 'Dec':'12' }

def Convert_Date(x):
    Year='20'+x[-2:]
    Month=month[x[-6:-3]]
    Day=x[:-7]
    return pd.to_datetime(Year+'-'+Month+'-'+Day)

# group data

df_group['Begin_Date']=df_group.begin_date.apply(lambda x: Convert_Date(x))
df_group['SubLine']= df_group.sub_line.apply(lambda x: int(x[14:]))
df_group['Area']= df_group.area.apply(lambda x: int(x[11:]))
group_used_cols=['group_id','Begin_Date','days','Area','SubLine','price']
df_order_1 = df_order.merge(df_group[group_used_cols], on='group_id')

# for order data

df_order_1['Order_Date']=df_order_1.order_date.apply(lambda x: Convert_Date(x))
df_order_1['Source_1']= df_order_1.source_1.apply(lambda x: int(x[11:]))
df_order_1['Source_2']= df_order_1.source_2.apply(lambda x: int(x[11:]))
df_order_1['Unit']= df_order_1.unit.apply(lambda x: int(x[11:]))
df_order_1['Begin_Date']=pd.to_datetime(df_order_1['Begin_Date'])
df_order_1['Order_Date']=pd.to_datetime(df_order_1['Order_Date'])
df_order_1['PreDays']=(df_order_1['Begin_Date']-df_order_1['Order_Date']).dt.days
df_order_1['Begin_Date_Weekday']= df_order_1['Begin_Date'].dt.dayofweek
df_order_1['Order_Date_Weekday']= df_order_1['Order_Date'].dt.dayofweek
df_order_1['Return_Date_Weekday']= (df_order_1['Begin_Date'].dt.dayofweek+df_order_1['days'])%7
order_used_columns=['order_id', 'group_id','Order_Date', 'Source_1', 'Source_2', 'Unit',
'people_amount', 'Begin_Date', 'days', 'Area', 'SubLine', 'price',
'PreDays','Begin_Date_Weekday', 'Order_Date_Weekday', 'Return_Date_Weekday']
df_order_2=df_order_1[order_used_columns]

# train/test data

df_train_1=df_train.merge(df_order_2,on='order_id')
df_test_1=df_test.merge(df_order_2,on='order_id')


In [5]:
print(df_order_1.shape,'\n\n')
print(df_order_1.columns,'\n\n')

print(df_order_2.shape,'\n\n')
print(df_order_2.columns,'\n\n')

# print(df_order_1.head())
# print(df_order_2.head())

(299995, 20) 


Index(['order_id', 'group_id', 'order_date', 'source_1', 'source_2', 'unit',
       'people_amount', 'Begin_Date', 'days', 'Area', 'SubLine', 'price',
       'Order_Date', 'Source_1', 'Source_2', 'Unit', 'PreDays',
       'Begin_Date_Weekday', 'Order_Date_Weekday', 'Return_Date_Weekday'],
      dtype='object') 


(299995, 16) 


Index(['order_id', 'group_id', 'Order_Date', 'Source_1', 'Source_2', 'Unit',
       'people_amount', 'Begin_Date', 'days', 'Area', 'SubLine', 'price',
       'PreDays', 'Begin_Date_Weekday', 'Order_Date_Weekday',
       'Return_Date_Weekday'],
      dtype='object') 




In [6]:
print(df_train_1.shape)
print(df_train_1.head())

(146620, 17)
   order_id  deal_or_not  group_id Order_Date  Source_1  Source_2  Unit  \
0     41571            1     53481 2017-03-14         1         1     4   
1     64125            1     53481 2017-03-27         2         1     5   
2     76131            0     53481 2017-04-19         2         2     6   
3    238671            0     53481 2017-04-19         2         2     6   
4     93305            1     41766 2017-05-03         2         1     5   

   people_amount Begin_Date  days  Area  SubLine   price  PreDays  \
0              4 2017-05-17    12     1        1  112900       64   
1              1 2017-05-17    12     1        1  112900       51   
2              2 2017-05-17    12     1        1  112900       28   
3              2 2017-05-17    12     1        1  112900       28   
4              1 2017-07-12    12     1        1  126900       70   

   Begin_Date_Weekday  Order_Date_Weekday  Return_Date_Weekday  
0                   2                   1               

In [7]:
print(df_test_1.shape)
print(df_test_1.head())

(150400, 17)
   order_id  deal_or_not  group_id Order_Date  Source_1  Source_2  Unit  \
0    136100          NaN     63695 2017-01-05         1         1     1   
1    140370          NaN     63695 2016-11-09         1         1     2   
2    211009          NaN     63695 2017-01-20         1         1     3   
3      5045          NaN     54305 2017-01-20         1         1     7   
4     37230          NaN     54305 2017-05-08         2         3     6   

   people_amount Begin_Date  days  Area  SubLine   price  PreDays  \
0              2 2017-03-22    12     1        1  106900       76   
1              1 2017-03-22    12     1        1  106900      133   
2              2 2017-03-22    12     1        1  106900       61   
3              6 2017-06-21    12     1        1  115900      152   
4              2 2017-06-21    12     1        1  115900       44   

   Begin_Date_Weekday  Order_Date_Weekday  Return_Date_Weekday  
0                   2                   3               

In [8]:
# save data to csv files 

df_train_1.to_csv("df_train_1.csv")
df_test_1.to_csv("df_test_1.csv")

In [9]:
# read data from the newly saved files

df_train_2 = pd.read_csv("df_train_1.csv")
df_test_2 = pd.read_csv("df_test_1.csv")

In [10]:
# check the newly imported data

df_train_2

,Unnamed: 0,order_id,deal_or_not,group_id,Order_Date,Source_1,Source_2,Unit,people_amount,Begin_Date,days,Area,SubLine,price,PreDays,Begin_Date_Weekday,Order_Date_Weekday,Return_Date_Weekday
0,0,41571,1,53481,2017-03-14,1,1,4,4,2017-05-17,12,1,1,112900,64,2,1,0
1,1,64125,1,53481,2017-03-27,2,1,5,1,2017-05-17,12,1,1,112900,51,2,0,0
2,2,76131,0,53481,2017-04-19,2,2,6,2,2017-05-17,12,1,1,112900,28,2,2,0
3,3,238671,0,53481,2017-04-19,2,2,6,2,2017-05-17,12,1,1,112900,28,2,2,0
4,4,93305,1,41766,2017-05-03,2,1,5,1,2017-07-12,12,1,1,126900,70,2,2,0
5,5,158994,1,41766,2017-04-10,1,1,10,4,2017-07-12,12,1,1,126900,93,2,0,0
6,6,180094,1,41766,2017-04-17,1,1,11,5,2017-07-12,12,1,1,126900,86,2,0,0
7,7,47901,0,41766,2017-04-10,1,1,12,2,2017-07-12,12,1,1,126900,93,2,0,0
8,8,102132,0,41766,2017-04-24,2,1,6,4,2017-07-12,12,1,1,126900,79,2,0,0
9,9,172968,0,41766,2017-06-13,1,1,13,3,2017-07-12,12,1,1,126900,29,2,1,0
